In [16]:
from typing import Type
import pyrtl
from pyrtl import *
import numpy as np
import onnx
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from enum import IntEnum
from hardware_accelerators.nn import load_model, MLP, softmax, get_pytorch_device
from hardware_accelerators.dtypes import *
from hardware_accelerators.rtllib import *
from hardware_accelerators.simulation import AcceleratorSimulator, CompiledSimulator
from hardware_accelerators.simulation.matrix_utils import *
from hardware_accelerators.simulation.buffer import WeightFIFOSimulator

np.random.seed(0)

In [74]:
CHUNK_SIZE = 4

config = AcceleratorConfig(
    array_size=CHUNK_SIZE,
    num_weight_tiles=4,
    data_type=BF16,
    weight_type=BF16,
    accum_type=BF16,
    pe_adder=float_adder,
    pe_multiplier=float_multiplier,
    pipeline=False,
    accum_adder=float_adder,
    accum_addr_width=12,
)

# New Accelerator Sim Testing


## GEMM


In [99]:
simulator = AcceleratorSimulator.default_config(
    array_size=3, num_weight_tiles=2, fast=False
)

simulator.setup()

weights = np.ones((3, 3))
activations = np.array([[-1, -2, -3], [-4, -5, -6], [7, 8, 9]])

simulator.load_weights(weights, 0)

simulator.execute_instruction(
    data_vec=activations[0],
    load_new_weights=True,
    flush_pipeline=False,
    activation_enable=True,
    activation_func="relu",
)
simulator.execute_instruction(
    data_vec=activations[1],
    accum_addr=1,
    flush_pipeline=False,
    activation_enable=True,
    activation_func="relu",
)
simulator.execute_instruction(
    data_vec=activations[2],
    accum_addr=2,
    activation_enable=True,
    activation_func="relu",
    flush_pipeline=True,
)

results = np.zeros((activations.shape[0], weights.shape[1]))

for i in range(3):
    simulator.execute_instruction(nop=True)
    results[i] = simulator.inspect_outputs()

gt = np.maximum(0, (activations @ weights))

assert np.isclose(results, gt).all()
print(results)

simulator.history

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [24. 24. 24.]]


[
 Simulation Step 0
 Input Signals:
 --------------------------------------------------------------------------------
   data_enable: 0
   data_inputs: None
   weight_start: 1
   weight_tile_addr: 0
   accum_addr: 0
   accum_mode: 0
   act_start: 0
   act_func: 0
 
 Systolic Array State:
 --------------------------------------------------------------------------------
 Inputs:
   w_en: 0
   enable: 0
   weights: [0. 0. 0.]
   data: [0. 0. 0.]
 
 Weights Matrix:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Data Matrix:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Accumulators:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Control Registers:
 data_controls: [0, [0, 0]]
 accum_controls: [0, 0, 0]
 control_out: 0
 
 Outputs:
 [0. 0. 0.]
 ----------------------------------------
 
 
 Accumulator State:
 --------------------------------------------------------------------------------
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 ReLU di

In [95]:
simulator.output_trace

[array([0., 0., 0.]), array([0., 0., 0.]), array([24., 24., 24.])]

## GEMV


In [ ]:
simulator = AcceleratorSimulator.default_config(array_size=3, num_weight_tiles=2)
simulator.setup()

w = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]]).T
activations = np.array([0, 0.5, 1])

simulator.load_weights(w, 0)

simulator.execute_instruction(
    data_vec=activations,
    load_new_weights=True,
    flush_pipeline=True,
    activation_enable=True,
    activation_func="relu",
)

print(f"Weights:\n{w}")
print(
    f"Systolic Array (weights loaded):\n{simulator.accelerator.inspect_systolic_array_state(simulator.sim)}"
)

simulator.execute_instruction(nop=True)
simulator.execute_instruction(nop=True)

results = simulator._get_outputs()
results

Weights:
[[1 2 3]
 [1 2 3]
 [1 2 3]]
Systolic Array (weights loaded):
Inputs:
  w_en: 0
  enable: 0
  weights: [0. 0. 0.]
  data: [0. 0. 0.]

Weights Matrix:
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]

Data Matrix:
[[0.  0.5 1. ]
 [0.5 1.  0. ]
 [1.  0.  0.5]]

Accumulators:
[[0.  1.  3. ]
 [0.5 3.  3. ]
 [0.  0.  0. ]]

Control Registers:
data_controls: [0, [0, 0]]
accum_controls: [0, 0, 1]
control_out: 0

Outputs:
[0. 0. 0.]
----------------------------------------



array([1.5, 3. , 4.5])

In [ ]:
simulator.history

[
 Simulation Step 0
 Input Signals:
 --------------------------------------------------------------------------------
   data_enable: 0
   data_inputs: None
   weight_start: 1
   weight_tile_addr: 0
   accum_addr: 0
   accum_mode: 0
   act_start: 0
   act_func: 0
 
 Systolic Array State:
 --------------------------------------------------------------------------------
 Inputs:
   w_en: 0
   enable: 0
   weights: [0. 0. 0.]
   data: [0. 0. 0.]
 
 Weights Matrix:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Data Matrix:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Accumulators:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Control Registers:
 data_controls: [0, [0, 0]]
 accum_controls: [0, 0, 0]
 control_out: 0
 
 Outputs:
 [0. 0. 0.]
 ----------------------------------------
 
 
 Accumulator State:
 --------------------------------------------------------------------------------
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Outputs

## Weight Loading


In [ ]:
# simulator = AcceleratorSimulator.default_config(array_size=3, num_weight_tiles=2)
simulator.setup()

w = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]])
w_neg = w * -1

simulator.load_weights(w, 0)
simulator.load_weights(w_neg, 1)

simulator.execute_instruction(load_new_weights=True, weight_tile_addr=0)
simulator.execute_instruction(load_new_weights=True, weight_tile_addr=1)
simulator.execute_instruction(nop=True)

for step in simulator.history:
    print(f"Step {step.step}\n{step.systolic_state.weights}\n")

Step 0
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Step 1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Step 2
[[3. 1. 2.]
 [0. 0. 0.]
 [0. 0. 0.]]

Step 3
[[2. 3. 1.]
 [3. 1. 2.]
 [0. 0. 0.]]

Step 4
[[1. 2. 3.]
 [2. 3. 1.]
 [3. 1. 2.]]

Step 5
[[1. 2. 3.]
 [2. 3. 1.]
 [3. 1. 2.]]

Step 6
[[-3. -1. -2.]
 [ 1.  2.  3.]
 [ 2.  3.  1.]]

Step 7
[[-2. -3. -1.]
 [-3. -1. -2.]
 [ 1.  2.  3.]]



# PyTorch Utilities


## Load MNIST Test Data


In [2]:
# Data transformation: convert images to tensor and normalize them
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
    ]
)
# Download MNIST test data
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


def get_batch(batch_size):
    loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    batch, labels = next(iter(loader))
    return batch.reshape(batch_size, -1).numpy(), labels.numpy()


def get_activation():
    image, _ = next(iter(test_loader))
    image = image.detach().numpy().reshape(-1)
    return image

In [103]:
get_batch(3)[0].T

array([[-0.42421296, -0.42421296, -0.42421296],
       [-0.42421296, -0.42421296, -0.42421296],
       [-0.42421296, -0.42421296, -0.42421296],
       ...,
       [-0.42421296, -0.42421296, -0.42421296],
       [-0.42421296, -0.42421296, -0.42421296],
       [-0.42421296, -0.42421296, -0.42421296]],
      shape=(784, 3), dtype=float32)

In [104]:
image, label = next(iter(test_loader))
image = image.numpy().reshape(-1)
image.shape, label

((784,), tensor([7]))

In [334]:
# Download MNIST test data
sample_dataset = datasets.MNIST(root="./data", train=False, download=False)
for i, (image, label) in enumerate(sample_dataset):
    display(image)
    if i > 10:
        break

## Initializing the trained PyTorch model


In [3]:
model = load_model("models/mlp_mnist.pth", "cpu")
model.eval()

/Users/kaibreese/UCSD/dsc180b/hardware-accelerators/hardware_accelerators/nn/util.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [11]:
for name, param in model.named_parameters():
    print(name)

fc1.weight
fc1.bias
fc2.weight
fc2.bias


### Numpy only inference


In [12]:
inputs = image  # numpy vector representing the image


# numpy softmax
def softmax(x: np.ndarray):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


fc1_weight = model.fc1.weight.data.numpy()
fc1_bias = model.fc1.bias.data.numpy()
fc2_weight = model.fc2.weight.data.numpy()
fc2_bias = model.fc2.bias.data.numpy()

x = inputs @ fc1_weight.T + fc1_bias
x = np.maximum(0, x)
x = x @ fc2_weight.T + fc2_bias
x = softmax(x)

# get the index of the maximum value
predicted_class = np.argmax(x)
print(f"Predicted class: {predicted_class}")

Predicted class: 7


### Generalized parameter/layer extraction


In [51]:
def get_layer_info(model):
    layers = []
    for name, module in model.named_modules():
        # Skip the root module itself
        if name == "":
            continue

        if isinstance(module, nn.Linear):
            layers.append(
                {
                    "type": "linear",
                    "weight": module.weight.data.numpy(),
                    "bias": (
                        module.bias.data.numpy() if module.bias is not None else None
                    ),
                }
            )
        elif isinstance(module, nn.ReLU):
            layers.append({"type": "relu"})
        elif isinstance(module, nn.Flatten):
            layers.append({"type": "flatten"})
        # Add more layer types as needed
        # elif isinstance(module, nn.Conv2d):
        #     layers.append(...)

    return layers


def numpy_inference(image, layers):
    x = image

    for layer in layers:
        if layer["type"] == "linear":
            x = np.dot(layer["weight"], x)
            if layer["bias"] is not None:
                x = x + layer["bias"]
        elif layer["type"] == "relu":
            x = np.maximum(0, x)
        elif layer["type"] == "flatten":
            # If input is already 1D, this is a no-op
            x = x.flatten()

    return x


# Extract model structure
layers = get_layer_info(model)

# Print model structure
for i, layer in enumerate(layers):
    if layer["type"] == "linear":
        print(
            f"Layer {i}: Linear - Weight shape: {layer['weight'].shape}, Bias shape: {layer['bias'].shape}"
        )
    else:
        print(f"Layer {i}: {layer['type']}")

# Run inference
prediction = numpy_inference(image, layers)
predicted_class = np.argmax(prediction)
predicted_class

Layer 0: flatten
Layer 1: Linear - Weight shape: (128, 784), Bias shape: (128,)
Layer 2: relu
Layer 3: Linear - Weight shape: (10, 128), Bias shape: (10,)


np.int64(7)

# Running model on the accelerator sim


## Setup Accelerator and config


In [ ]:
simulator = AcceleratorSimulator(config)

In [64]:
sim_logprobs = simulator.simulate_pytorch_model(model, get_activation())
sim_logprobs, np.argmax(sim_logprobs)

fc1_weight: (128, 784)
fc1_bias: (128,)
fc2_weight: (10, 128)
fc2_bias: (10,)
inputs: (784,)
fc1_out: (128,)


(array([8.82158504e-07, 8.82158504e-07, 8.82158504e-07, 3.37657267e-03,
        1.00112238e-09, 3.99896932e-08, 8.47857112e-11, 9.96611666e-01,
        3.75668401e-08, 7.27170995e-06, 8.82158504e-07, 8.82158504e-07]),
 np.int64(7))

## Extract weight and bias matrices from model


In [7]:
# Extract layer weights and biases
weights_1 = model.fc1.weight.numpy(force=True)
bias_1 = model.fc1.bias.numpy(force=True)
weights_2 = model.fc2.weight.numpy(force=True)
bias_2 = model.fc2.bias.numpy(force=True)

# Add bias to first layer weights and 1 to activations
W_aug, x_aug = bias_trick(weights_1, bias_1, get_activation())

# Run the first layer
x1 = W_aug @ x_aug
act_1 = np.maximum(x1, 0)

# Add bias to second layer weights and 1 to previous output
W2_aug, x2_aug = bias_trick(weights_2, bias_2, act_1)

# Run the second layer
logits = W2_aug @ x2_aug
act_out2 = softmax(logits)
np.argmax(act_out2)

np.int64(7)

In [8]:
testsize = 10000
testsize.bit_length()
2**16

65536

## Verify the tile generator for GEMV


In [15]:
partial = np.zeros(config.array_size)
result = []
tile_generator = generate_gemv_tiles(x_aug, W_aug, CHUNK_SIZE)

for tile in tile_generator:
    if tile.first:
        partial = tile.matrix @ tile.vector
    else:
        partial += tile.matrix @ tile.vector

    if tile.last:
        result.append(partial)

result = np.array(result).flatten()

np.isclose(result, x1).all()

np.True_

## Test simulator with tile generator


### Partial output test of first hidden layer


In [18]:
simulator.setup()
# chunk_size = 16
chunk_size = config.array_size
tile_generator = generate_gemv_tiles(x_aug, W_aug, chunk_size)

result = []

for tile in tile_generator:
    # partial = tile.matrix @ tile.vector
    partial = tile.vector @ tile.matrix
    if tile.index >= len(result):
        result.append(partial)
    else:
        result[tile.index] += partial

    simulator.load_weights(weights=tile.matrix, tile_addr=0)
    simulator.execute_instruction(
        load_new_weights=True,
        weight_tile_addr=0,
        data_vec=tile.vector,
        accum_addr=tile.index,
        accum_mode=not tile.first,
        activation_func="relu",
        activation_enable=tile.last,
        flush_pipeline=True,
    )

    simulator.step()
    sim_partial = simulator.accelerator.inspect_accumulator_state(simulator.sim)[
        tile.index
    ]

    print(tile)
    print(f"Calculated partial:\n{result[tile.index]}\n")
    print(f"Simulation Partial Result:\n{sim_partial}\n\n")

    if tile.last:
        break

GemvTile(
    index: 0
    first: True
    last: False
    matrix:
[[ 0.0067 -0.0223  0.0327 -0.0118]
 [ 0.0036  0.017   0.0161  0.018 ]
 [ 0.0055  0.018  -0.002  -0.0071]
 [-0.0195 -0.0147  0.0034 -0.0122]]
    vector:
[-0.4242 -0.4242 -0.4242 -0.4242]
    partial_result:
[ 0.0016  0.0008 -0.0213  0.0056]
)
Calculated partial:
[ 0.0015598   0.00084817 -0.02133417  0.00556452]

Simulation Partial Result:
[ 0.00158691  0.00091553 -0.02111816  0.0055542 ]


GemvTile(
    index: 0
    first: False
    last: False
    matrix:
[[ 0.0382 -0.0016  0.0227  0.0357]
 [-0.0001  0.0306 -0.0311  0.0168]
 [-0.0326 -0.0403 -0.0299 -0.0426]
 [ 0.018   0.0075 -0.0182 -0.0209]]
    vector:
[-0.4242 -0.4242 -0.4242 -0.4242]
    partial_result:
[-0.01    0.0016  0.0239  0.0047]
)
Calculated partial:
[-0.00840414  0.00243366  0.00258482  0.01024831]

Simulation Partial Result:
[-0.00842285  0.00245667  0.00256348  0.02416992]


GemvTile(
    index: 0
    first: False
    last: False
    matrix:
[[-0.0205  

### Run full model


In [ ]:
simulator.setup()
chunk_size = config.array_size

tile_generator = generate_gemv_tiles(x_aug, W_aug, CHUNK_SIZE)

for tile in tile_generator:
    simulator.load_weights(weights=tile.matrix.T, tile_addr=0)
    simulator.execute_instruction(
        load_new_weights=True,
        weight_tile_addr=0,
        data_vec=tile.vector,
        accum_addr=tile.index,
        accum_mode=not tile.first,
        activation_func="relu",
        activation_enable=tile.last,
        flush_pipeline=True,
    )

simulator.execute_instruction(nop=True)
simulator.execute_instruction(nop=True)

sim_fc1 = np.array(simulator.output_trace)

# simulator.reset_output_trace()
simulator.output_trace = []

W2_aug, fc1_aug = bias_trick(weights_2, bias_2, sim_fc1.flatten())

fc2_tile_generator = generate_gemv_tiles(fc1_aug, W2_aug, CHUNK_SIZE)

for tile in fc2_tile_generator:
    simulator.load_weights(weights=tile.matrix.T, tile_addr=0)
    simulator.execute_instruction(
        load_new_weights=True,
        weight_tile_addr=0,
        data_vec=tile.vector,
        accum_addr=tile.index,
        accum_mode=not tile.first,
        activation_enable=tile.last,
        flush_pipeline=True,
    )

simulator.execute_instruction(nop=True)
simulator.execute_instruction(nop=True)

sim_fc2 = np.array(simulator.output_trace).flatten()
sim_fc2

array([[-1.484375  , -2.34375   ,  3.078125  , -1.5859375 ],
       [ 0.        ,  6.8125    ,  6.53125   ,  0.        ],
       [ 0.        ,  2.875     ,  3.3125    ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  3.6875    ,  0.        ],
       [ 0.        ,  0.        ,  4.21875   ,  7.46875   ],
       [ 0.        ,  0.        ,  0.        ,  2.484375  ],
       [ 0.        ,  4.625     ,  0.72265625,  1.015625  ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 5.09375   ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  9.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  7.53125   ],
       [ 0.        ,  6.5625    ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.

# Testing the compiled simulator


In [17]:
reset_working_block()
compsim = CompiledSimulator(config)

In [18]:
compsim.reset_output_trace()

for tile in generate_gemv_tiles(x_aug, W_aug, config.array_size):
    compsim.execute_instruction(
        weights=tile.matrix.T,
        data=tile.vector.reshape(1, -1),
        accum_addr=tile.index,
        accum_mode=not tile.first,
        activation_func="relu",
        activation_enable=tile.last,
        flush_pipeline=True,
    )

compsim.step()
compsim.step()

x1 = np.array(compsim.output_trace).flatten()

compsim.reset_output_trace()

W2_aug, x1_aug = bias_trick(weights_2, bias_2, x1)

for tile in generate_gemv_tiles(x1_aug, W2_aug, config.array_size):
    compsim.execute_instruction(
        weights=tile.matrix.T,
        data=tile.vector.reshape(1, -1),
        accum_addr=tile.index,
        accum_mode=not tile.first,
        activation_func="relu",
        activation_enable=tile.last,
        flush_pipeline=True,
    )

compsim.step()
compsim.step()

logits = np.array(compsim.output_trace).flatten()
logprobs = softmax(logits)
logprobs, np.argmax(logprobs)

(array([8.77499496e-07, 8.77499496e-07, 8.77499496e-07, 8.05719775e-03,
        5.83661448e-04, 8.77499496e-07, 8.77499496e-07, 9.91348188e-01,
        8.77499496e-07, 3.93267990e-06, 8.77499496e-07, 8.77499496e-07]),
 np.int64(7))

## Continuous Batching


### Tiling Generator Function


In [41]:
@dataclass
class BatchGemmTile:
    """Represents a tile for batch matrix multiplication.

    Attributes:
        output_row_idx: Starting row index in output matrix where results should be accumulated
        weight_tile: Square tile from weight matrix of shape (chunk_size, chunk_size)
        batch_tile: Batch of input vectors of shape (chunk_size, batch_size)
        first: True if this is the first tile for the current output rows
        last: True if this is the last tile for the current output rows
    """

    output_row_idx: int
    weight_tile: np.ndarray
    batch_tile: np.ndarray
    first: bool
    last: bool


def generate_batch_gemm_tiles(
    weights: np.ndarray, batch_inputs: np.ndarray, chunk_size: int
) -> Generator[BatchGemmTile, None, None]:
    """Generates tiles for batch matrix multiplication optimized for systolic array processing.

    Args:
        weights: Weight matrix of shape (output_dim, input_dim)
        batch_inputs: Input batch of shape (input_dim, batch_size)
        chunk_size: Size of systolic array (NxN)

    Yields:
        BatchGemmTile objects containing chunked weights and corresponding batch inputs
    """
    out_dim, in_dim = weights.shape
    in_dim2, batch_size = batch_inputs.shape
    assert in_dim == in_dim2, "Weight and input dimensions must match"

    # Calculate padding needed for weights
    pad_out = (chunk_size - out_dim % chunk_size) % chunk_size
    pad_in = (chunk_size - in_dim % chunk_size) % chunk_size

    # Pad weights and inputs
    padded_weights = np.pad(weights, ((0, pad_out), (0, pad_in)))
    padded_inputs = np.pad(batch_inputs, ((0, pad_in), (0, 0)))

    # Calculate number of chunks
    num_out_chunks = (out_dim + pad_out) // chunk_size
    num_in_chunks = (in_dim + pad_in) // chunk_size

    # Generate tiles
    for i in range(num_out_chunks):  # Output dimension chunks
        for j in range(num_in_chunks):  # Input dimension chunks
            # Extract weight tile
            w_row_start = i * chunk_size
            w_row_end = (i + 1) * chunk_size
            w_col_start = j * chunk_size
            w_col_end = (j + 1) * chunk_size
            weight_tile = padded_weights[w_row_start:w_row_end, w_col_start:w_col_end]

            # Extract corresponding batch input tile
            batch_start = j * chunk_size
            batch_end = (j + 1) * chunk_size
            batch_tile = padded_inputs[batch_start:batch_end, :]

            is_last_input_chunk = j == num_in_chunks - 1
            is_first_input_chunk = j == 0

            yield BatchGemmTile(
                output_row_idx=i,
                weight_tile=weight_tile,
                batch_tile=batch_tile,
                first=is_first_input_chunk,
                last=is_last_input_chunk,
            )

### Examples


In [ ]:
def test_batch_gemm_tiles():
    # Create a weight matrix 8x6 with letters
    weights = np.array(
        [
            ["A1", "A2", "A3", "A4", "A5", "A6"],
            ["B1", "B2", "B3", "B4", "B5", "B6"],
            ["C1", "C2", "C3", "C4", "C5", "C6"],
            ["D1", "D2", "D3", "D4", "D5", "D6"],
            ["E1", "E2", "E3", "E4", "E5", "E6"],
            ["F1", "F2", "F3", "F4", "F5", "F6"],
            ["G1", "G2", "G3", "G4", "G5", "G6"],
            ["H1", "H2", "H3", "H4", "H5", "H6"],
        ]
    )

    # Create batch inputs 6x4 with numbers
    # Each column represents one input in the batch
    batch_inputs = np.array(
        [
            ["x1", "y1", "z1", "w1"],
            ["x2", "y2", "z2", "w2"],
            ["x3", "y3", "z3", "w3"],
            ["x4", "y4", "z4", "w4"],
            ["x5", "y5", "z5", "w5"],
            ["x6", "y6", "z6", "w6"],
        ]
    )

    # Use a chunk size of 4
    chunk_size = 2

    print("Original Weight Matrix (8x6):")
    print(weights)
    print("\nOriginal Batch Inputs (6x4):")
    print(batch_inputs)
    print("\nChunk Size:", chunk_size)
    print("\nGenerated Tiles:")
    print("---------------")

    for i, tile in enumerate(
        generate_batch_gemm_tiles(weights, batch_inputs, chunk_size)
    ):
        print(f"\nTile {i + 1}")
        print(f"Output Row Index: {tile.output_row_idx}")
        print(f"First: {tile.first}, Last: {tile.last}")
        print("\nWeight Tile (4x4):")
        print(tile.weight_tile)
        print("\nBatch Tile (4x4):")
        print(tile.batch_tile)
        print("\n" + "=" * 50)


# Run the test
test_batch_gemm_tiles()

Original Weight Matrix (8x6):
[['A1' 'A2' 'A3' 'A4' 'A5' 'A6']
 ['B1' 'B2' 'B3' 'B4' 'B5' 'B6']
 ['C1' 'C2' 'C3' 'C4' 'C5' 'C6']
 ['D1' 'D2' 'D3' 'D4' 'D5' 'D6']
 ['E1' 'E2' 'E3' 'E4' 'E5' 'E6']
 ['F1' 'F2' 'F3' 'F4' 'F5' 'F6']
 ['G1' 'G2' 'G3' 'G4' 'G5' 'G6']
 ['H1' 'H2' 'H3' 'H4' 'H5' 'H6']]

Original Batch Inputs (6x4):
[['x1' 'y1' 'z1' 'w1']
 ['x2' 'y2' 'z2' 'w2']
 ['x3' 'y3' 'z3' 'w3']
 ['x4' 'y4' 'z4' 'w4']
 ['x5' 'y5' 'z5' 'w5']
 ['x6' 'y6' 'z6' 'w6']]

Chunk Size: 2

Generated Tiles:
---------------

Tile 1
Output Row Index: 0
First: True, Last: False

Weight Tile (4x4):
[['A1' 'A2']
 ['B1' 'B2']]

Batch Tile (4x4):
[['x1' 'y1' 'z1' 'w1']
 ['x2' 'y2' 'z2' 'w2']]


Tile 2
Output Row Index: 0
First: False, Last: False

Weight Tile (4x4):
[['A3' 'A4']
 ['B3' 'B4']]

Batch Tile (4x4):
[['x3' 'y3' 'z3' 'w3']
 ['x4' 'y4' 'z4' 'w4']]


Tile 3
Output Row Index: 0
First: False, Last: True

Weight Tile (4x4):
[['A5' 'A6']
 ['B5' 'B6']]

Batch Tile (4x4):
[['x5' 'y5' 'z5' 'w5']
 ['x6' 'y6

In [ ]:
def test_batch_gemm_tiles_with_computation():
    # Set random seed for reproducibility
    np.random.seed(42)

    # Create example dimensions
    output_dim = 8
    input_dim = 6
    batch_size = 4
    chunk_size = 4

    # Create random matrices with small integers for easy verification
    weights = np.random.randint(1, 5, size=(output_dim, input_dim))
    batch_inputs = np.random.randint(1, 5, size=(input_dim, batch_size))

    print("Weight Matrix (8x6):")
    print(weights)
    print("\nBatch Inputs (6x4):")
    print(batch_inputs)
    print("\nChunk Size:", chunk_size)

    # Expected result using direct matrix multiplication
    expected_result = weights @ batch_inputs

    print("\nExpected Result (8x4):")
    print(expected_result)

    # Initialize accumulator for our tiled results
    # Shape matches expected_result: (output_dim, batch_size)
    accumulated_result = np.zeros((output_dim, batch_size))

    print("\nProcessing Tiles:")
    print("----------------")

    for i, tile in enumerate(
        generate_batch_gemm_tiles(weights, batch_inputs, chunk_size)
    ):
        print(f"\nTile {i + 1}")
        print(f"Output Row Index: {tile.output_row_idx}")
        print(f"First: {tile.first}, Last: {tile.last}")

        print("\nWeight Tile:")
        print(tile.weight_tile)
        print("\nBatch Tile:")
        print(tile.batch_tile)

        # Compute partial result for this tile
        partial_result = tile.weight_tile @ tile.batch_tile
        print("\nPartial Result:")
        print(partial_result)

        # Accumulate result in the correct position
        row_start = tile.output_row_idx * chunk_size
        row_end = min(row_start + chunk_size, output_dim)
        if tile.first:
            accumulated_result[row_start:row_end, :] = partial_result[
                : row_end - row_start, :
            ]
        else:
            accumulated_result[row_start:row_end, :] += partial_result[
                : row_end - row_start, :
            ]

        print("\nAccumulated Result so far:")
        print(accumulated_result)
        print("\n" + "=" * 50)

    # Verify results
    print("\nFinal Accumulated Result:")
    print(accumulated_result)
    print("\nExpected Result:")
    print(expected_result)
    print("\nResults Match:", np.allclose(accumulated_result, expected_result))

    # Show difference if any
    if not np.allclose(accumulated_result, expected_result):
        print("\nDifference:")
        print(accumulated_result - expected_result)


# Run the test
test_batch_gemm_tiles_with_computation()

Weight Matrix (8x6):
[[3 4 1 3 3 4]
 [1 1 3 2 3 3]
 [3 3 4 1 4 4]
 [4 3 2 1 2 4]
 [4 2 2 2 4 4]
 [1 1 4 2 2 1]
 [4 1 1 3 3 3]
 [2 4 4 4 4 3]]

Batch Inputs (6x4):
[[2 2 3 2]
 [3 4 3 4]
 [4 1 3 1]
 [3 3 1 1]
 [3 2 4 1]
 [4 2 2 2]]

Chunk Size: 4

Expected Result (8x4):
[[56 46 47 37]
 [44 27 35 20]
 [62 41 55 35]
 [50 37 44 33]
 [56 40 50 32]
 [37 22 30 16]
 [45 34 39 25]
 [68 50 56 38]]

Processing Tiles:
----------------

Tile 1
Output Row Index: 0
First: True, Last: False

Weight Tile:
[[3 4 1 3]
 [1 1 3 2]
 [3 3 4 1]
 [4 3 2 1]]

Batch Tile:
[[2 2 3 2]
 [3 4 3 4]
 [4 1 3 1]
 [3 3 1 1]]

Partial Result:
[[31 32 27 26]
 [23 15 17 11]
 [34 25 31 23]
 [28 25 28 23]]

Accumulated Result so far:
[[31. 32. 27. 26.]
 [23. 15. 17. 11.]
 [34. 25. 31. 23.]
 [28. 25. 28. 23.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


Tile 2
Output Row Index: 0
First: False, Last: True

Weight Tile:
[[3 4 0 0]
 [3 3 0 0]
 [4 4 0 0]
 [2 4 0 0]]

Batch Tile:
[[3 2 4 1]
 [4 2 2 

### Testing the compiled sim with batches


In [59]:
BATCH_SIZE = 3

batch, labels = get_batch(BATCH_SIZE)

wb, ab = bias_trick(weights_1, bias_1, batch)

wb.shape, ab.shape

((128, 785), (3, 785))

In [60]:
batch.shape

(3, 784)

In [69]:
reset_working_block()
compsim = CompiledSimulator(
    AcceleratorConfig(
        array_size=8,
        num_weight_tiles=1,
        # data_type=Float8,
        # weight_type=Float8,
        # accum_type=Float8,
        data_type=BF16,
        weight_type=BF16,
        accum_type=BF16,
        pe_adder=float_adder,
        # pe_multiplier=lmul_fast,
        pe_multiplier=float_multiplier,
        pipeline=False,
        accum_adder=float_adder,
        accum_addr_width=8,
    )
)

In [70]:
batch, labels = get_batch(8)

sim_probs = compsim.run_mlp_batch(model, batch)
sim_preds = np.argmax(sim_probs, axis=1)
correct = sim_preds == labels

print(f"Accuracy: {np.sum(correct)/len(correct)}")
print(f"Correct: {correct}")

Completed 1584/1618 tiles
Final output shape: (8, 10)
Accuracy: 1.0
Correct: [ True  True  True  True  True  True  True  True]


In [32]:
bias_1.shape

(128,)

In [28]:
sim_probs

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [ ]:
model = load_model("models/mlp_mnist.pth")  # .bfloat16()
device = get_pytorch_device()
loader = DataLoader(test_dataset, batch_size=10, shuffle=False)
batch, labels = next(iter(loader))
batch = batch.to(device)
labels = labels.to(device)

/Users/kaibreese/UCSD/dsc180b/hardware-accelerators/hardware_accelerators/nn/util.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model

In [13]:
batch

tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.424

In [4]:
preds = model.forward(batch)

In [6]:
preds = torch.argmax(torch.softmax(preds, dim=1), dim=1)
preds == labels

tensor([True, True, True, True, True, True, True, True, True, True],
       device='mps:0')

### Extra Testing


In [33]:
batch_in = np.array(
    [
        [0.01, 0.02, 0.03, 0.04, 0.05],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [1, 2, 3, 4, 5],
        [10, 20, 30, 40, 50],
    ]
)

fake_weights = np.array(
    [
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4],
        [5, 5, 5, 5, 5],
        # [6, 6, 6, 6, 6],
    ]
)

np.array(
    [
        [1, 2, 3, 4, 5, 6],
        [1, 2, 3, 4, 5, 6],
        [1, 2, 3, 4, 5, 6],
        [1, 2, 3, 4, 5, 6],
        [1, 2, 3, 4, 5, 6],
    ]
)

# fake_weights @ batch_in.T
batch_in @ fake_weights.T

array([[1.5e-01, 3.0e-01, 4.5e-01, 6.0e-01, 7.5e-01],
       [1.5e+00, 3.0e+00, 4.5e+00, 6.0e+00, 7.5e+00],
       [1.5e+01, 3.0e+01, 4.5e+01, 6.0e+01, 7.5e+01],
       [1.5e+02, 3.0e+02, 4.5e+02, 6.0e+02, 7.5e+02]])

In [34]:
results = []
partials = []
final = []

for tile in generate_batch_gemm_tiles(fake_weights, batch_in.T, 3):
    w = tile.weight_tile.T
    b = tile.batch_tile.T
    output = b @ w

    partials.append(output.copy())
    if tile.first:
        results.append(output)
    else:
        results[-1] += output

    if tile.last:
        part_list = np.array(partials)
        print(f"Partials:\n{part_list}\n")
        print(f"Partial result {tile.output_row_idx}")
        print(results[-1])
        print("=" * 30, "\n")
        partials = []

    # print(f"Index {tile.output_row_idx}\n")
    # print(f"Weights:\n{w}\n")
    # print(f"Batch:\n{b}\n")
    # print(f"Output:\n{output}\n")
    # print("="*30, "\n")

Partials:
[[[6.0e-02 1.2e-01 1.8e-01]
  [6.0e-01 1.2e+00 1.8e+00]
  [6.0e+00 1.2e+01 1.8e+01]
  [6.0e+01 1.2e+02 1.8e+02]]

 [[9.0e-02 1.8e-01 2.7e-01]
  [9.0e-01 1.8e+00 2.7e+00]
  [9.0e+00 1.8e+01 2.7e+01]
  [9.0e+01 1.8e+02 2.7e+02]]]

Partial result 0
[[1.5e-01 3.0e-01 4.5e-01]
 [1.5e+00 3.0e+00 4.5e+00]
 [1.5e+01 3.0e+01 4.5e+01]
 [1.5e+02 3.0e+02 4.5e+02]]

Partials:
[[[2.4e-01 3.0e-01 0.0e+00]
  [2.4e+00 3.0e+00 0.0e+00]
  [2.4e+01 3.0e+01 0.0e+00]
  [2.4e+02 3.0e+02 0.0e+00]]

 [[3.6e-01 4.5e-01 0.0e+00]
  [3.6e+00 4.5e+00 0.0e+00]
  [3.6e+01 4.5e+01 0.0e+00]
  [3.6e+02 4.5e+02 0.0e+00]]]

Partial result 1
[[6.0e-01 7.5e-01 0.0e+00]
 [6.0e+00 7.5e+00 0.0e+00]
 [6.0e+01 7.5e+01 0.0e+00]
 [6.0e+02 7.5e+02 0.0e+00]]



In [72]:
reset_working_block()
compsim = CompiledSimulator(
    AcceleratorConfig(
        array_size=3,
        num_weight_tiles=1,
        # data_type=Float8,
        # weight_type=Float8,
        data_type=BF16,
        weight_type=BF16,
        accum_type=BF16,
        pe_adder=float_adder,
        # pe_multiplier=lmul_fast,
        pe_multiplier=float_multiplier,
        pipeline=False,
        accum_adder=float_adder,
        accum_addr_width=4,
    )
)

#### Writing the compiled simulation code to a file

> might be useful for running demos if we precompile simulations for common configs


In [ ]:
with open(f"{compsim.sim._dir}/pyrtlsim.c", "r") as f:
    with open("pyrtlsim.c", "w") as f2:
        f2.write(f.read())

In [ ]:
compsim.reset_output_trace()
hidden_chunks = []

for tile in generate_batch_gemm_tiles(
    fake_weights * -1, batch_in.T, compsim.config.array_size
):
    compsim.execute_instruction(
        weights=tile.weight_tile.T,
        data=tile.batch_tile.T,
        accum_addr=0,
        accum_mode=not tile.first,
        activation_func="relu",
        activation_enable=tile.last,
        flush_pipeline=True,
    )

    if tile.last:
        compsim.step()
        compsim.step()
        results = np.array(compsim.output_trace)
        print(f"Partial result {tile.output_row_idx}")
        # print(results)
        print_arrays(results)
        hidden_chunks.append(results)
        compsim.reset_output_trace()

# Concatenate chunks and slice
hidden_out = np.concatenate(hidden_chunks, axis=1)[:, : fake_weights.shape[0]]
print(f"\nFinal Output:")
print_arrays(hidden_out)

Partial result 0
Array of shape (4, 3):
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Partial result 1
Array of shape (4, 3):
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


Final Output:
Array of shape (4, 5):
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]



In [ ]:
def gemm(A, B):
    """computes A @ B using compiled simulator"""
    compsim.reset_output_trace()
    output_chunks = []
    accum_chunks = []
    for tile in generate_batch_gemm_tiles(A, B, compsim.config.array_size):
        compsim.execute_instruction(
            weights=tile.weight_tile.T,
            data=tile.batch_tile.T,
            accum_addr=0,
            accum_mode=0 if tile.first else 1,
            activation_func="relu",
            activation_enable=tile.last,
            flush_pipeline=True,
        )

        if tile.last:
            compsim.step()
            compsim.step()
            compsim.step()
            results = np.array(compsim.output_trace)
            output_chunks.append(results)
            compsim.reset_output_trace()

            accum_mem = compsim.inspect_accumulator_mem()[: B.shape[1]]
            accum_chunks.append(accum_mem)

            # return results, np.array(accum_chunks)[0].T

    # # Concatenate chunks and slice
    output_chunks = np.concatenate(output_chunks, axis=1)[:, : A.shape[0]]
    return output_chunks.T, np.array(accum_chunks)


np.random.seed(0)
A = np.random.randint(-10, 10, (10, 3))
B = np.random.rand(3, 5)

AB, accum_chunks = gemm(A, B)

gt = torch.from_numpy(A).type(torch.bfloat16) @ torch.from_numpy(B).type(torch.bfloat16)
gt = torch.relu(gt)

print_arrays(W=A, A=B, accumulator=accum_chunks, sim=AB, gt=gt)

W (10, 3):
[[  2   5 -10]
 [ -7  -7  -3]
 [ -1   9   8]
 [ -6  -4   2]
 [ -9  -4  -3]
 [  4   7  -5]
 [  3  -2  -1]
 [  9   6   9]
 [ -5   5   5]
 [-10   8  -7]]

A (3, 5):
[[0.799 0.461 0.781 0.118 0.64 ]
 [0.143 0.945 0.522 0.415 0.265]
 [0.774 0.456 0.568 0.019 0.618]]

accumulator (4, 5, 3):
[[[ -5.438  -8.875   6.688]
  [  1.062 -11.125  11.688]
  [ -1.531 -10.75    8.438]
  [  2.125  -3.766   3.766]
  [ -3.562  -8.125   6.688]]

 [[ -3.812 -10.      0.312]
  [ -5.625  -9.25    6.156]
  [ -5.594 -10.75    3.922]
  [ -2.328  -2.766   3.266]
  [ -3.656  -8.625   1.312]]

 [[  1.344  14.938   0.605]
  [ -0.953  13.875   4.688]
  [  0.734  15.188   1.531]
  [ -0.494   3.703   1.562]
  [  0.766  12.812   1.211]]

 [[-12.25    0.      0.   ]
  [ -0.281   0.      0.   ]
  [ -7.594   0.      0.   ]
  [  2.016   0.      0.   ]
  [ -8.562   0.      0.   ]]]

sim (10, 5):
[[ 0.     1.062  0.     2.125  0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 6.688 11.688  8.438  3.766  6.688]
 [ 0.  

In [ ]:
A[0] @ B[:, -1]
A[0], B[:, -1]

(array([  2,   5, -10]), array([0.63992102, 0.26455561, 0.6176355 ]))

In [ ]:
compsim.execute_instruction(weights=A[0], data=B[:, -1], accum_addr=69)
compsim.inspect_accumulator_mem()[69]

AssertionError: Weights must be 4x4